In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# Load the OHLC data
df = pd.read_csv('TCS.NS.csv')

# Extract the input (X) and output (y) variables for prediction
X = df[['Open', 'High', 'Low', 'Volume', 'Close']]
y = df['Close'].shift(-1)

# Remove the last row since it contains NaN for the shifted output
X = X.iloc[:-1]
y = y.iloc[:-1]

# Scale the input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_scaled.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit the LSTM model on the entire data
model.fit(X_reshaped, y, epochs=50, batch_size=32, verbose=0)

# Predict the closing value of the stock for the next 10,000 days
last_input = X_reshaped[-1]  # get the last input for prediction
predictions = []
for i in range(10000):
    next_close_scaled = model.predict(np.array([last_input]))[0][0]  # use the LSTM model to predict the next closing value
    next_close = scaler.inverse_transform([[next_close_scaled]])[0][0]  # scale back the predicted value
    next_date = datetime.strptime(df['Date'].iloc[-1], '%Y-%m-%d') + timedelta(days=1)  # compute the next date
    predictions.append(next_close)
    
    # Create the next input for the next iteration
    next_input = np.concatenate((last_input[1:], np.array([[next_close_scaled]])))  # update the last 'Close' value with the predicted value
    last_input = next_input
    
# Store the predictions in a new CSV file
output_df = pd.DataFrame({'Date': pd.date_range(start=df['Date'].iloc[-1], periods=10000, freq='D')[1:],
                          'Predicted_Close': predictions})
output_df.to_csv('predictions.csv', index=False)

# Print the predictions for the first 10,000 days
for i in range(10000):
    print('The predicted closing value of the stock for', output_df['Date'].iloc[i].date(), 'is:', predictions[i])


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,5)

In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import datetime, timedelta

# Load the OHLC data
df = pd.read_csv('TCS.NS.csv')

# Extract the input (X) and output (y) variables for prediction
X = df[['Open', 'High', 'Low', 'Close']]
y = df['Close']

# Scale the input data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Reshape the input data for LSTM
X_reshaped = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(X_scaled.shape[1], 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Fit the LSTM model on the entire data
model.fit(X_reshaped, y, epochs=50, batch_size=32, verbose=0)

# Predict the closing value of the stock for the next 10,000 days
last_input = X_reshaped[-1]  # get the last input for prediction
predictions = []
for i in range(10000):
    next_close_scaled = model.predict(np.array([last_input]))[0][0]  # use the LSTM model to predict the next closing value
    next_close = scaler.inverse_transform([[next_close_scaled]])[0][0]  # scale back the predicted value
    next_date = datetime.strptime(df['Date'].iloc[-1], '%Y-%m-%d') + timedelta(days=1)  # compute the next date
    predictions.append(next_close)
    
    # Create the next input for the next iteration
    next_input = np.concatenate((last_input[1:], np.array([[next_close_scaled]])))  # update the last 'Close' value with the predicted value
    last_input = next_input

# Create a DataFrame with predicted dates and close values
predicted_dates = pd.date_range(start=df['Date'].iloc[-1], periods=10000, freq='D')[1:]
output_df = pd.DataFrame({'Date': predicted_dates, 'Predicted_Close': predictions})

# Save the predictions to a new CSV file
output_df.to_csv('predicted_close.csv', index=False)


ValueError: non-broadcastable output operand with shape (1,1) doesn't match the broadcast shape (1,4)